# Tutorial 4: Save and load Function

## Overview

While constructing and using a flow, it's convenient to introspect the inner architecture of the flow to understand how it work and to ensure that we plug-n-play the right component into the flow. This capability can prove even more usefulness for those who don't build the flow and just use those that are packaged and distributed by other people.

Any flow constructed with `theflow.Function` allow users to:

- View values of all current nodes and params.
- View the definition of any nodes and params.
- Check if any object is compatible with the flow and can be plugged into the flow.
- Visualize the flow architecture.

Suppose we have a flow `Plus` that consists of component `Sum1` and `Sum2` as follow:

In [1]:
import yaml
from pprint import pprint

from theflow import Function, Param, Node, load


def callback(obj, type_):
    return obj.a * 2


class Sum1(Function):
    a: int
    b: int = 10
    c: int = 10
    d: int = Param(default_callback=lambda obj: obj.b * 2)

    def run(self) -> int:
        return self.a + self.b + self.c


class Sum2(Function):
    a: int

    def run(self, a, b: int, *args, **kwargs) -> int:
        return self.a + a + b


class Plus(Function):
    a: int
    e: int
    x: Function
    y: Function = Node(default=Sum1.withx(a=100))
    m: Function = Node(default=Sum2.withx(a=100))

    @Param.auto()
    def f(self):
        return self.a + self.e

    def run(self) -> int:
        x, y, m = self.x(), self.y(), self.m()
        print(f"{x=}, {y=}, {z=}")
        return x + y + z

In [2]:
step = Plus(a=20, e=20, x=Sum1(a=20))
print(yaml.dump(step.dump(), sort_keys=False))

type: __main__.Plus
params:
  a: 20
  e: 20
  f: 40
nodes:
  m:
    type: __main__.Sum2
    params:
      a: 100
    nodes: {}
  x:
    type: __main__.Sum1
    params:
      a: 20
      b: 10
      c: 10
      d: 20
    nodes: {}
  y:
    type: __main__.Sum1
    params:
      a: 100
      b: 10
      c: 10
      d: 20
    nodes: {}



In [3]:
modules = {
    "__main__.Plus": Plus,
    "__main__.Sum2": Sum2,
    "__main__.Sum1": Sum1,
}

step2 = load(step.dump(), safe=True, allowed_modules=modules)

In [4]:
step2.dump()

{'type': '__main__.Plus',
 'params': {'a': 20, 'e': 20, 'f': 40},
 'nodes': {'m': {'type': '__main__.Sum2', 'params': {'a': 100}, 'nodes': {}},
  'x': {'type': '__main__.Sum1',
   'params': {'a': 20, 'b': 10, 'c': 10, 'd': 20},
   'nodes': {}},
  'y': {'type': '__main__.Sum1',
   'params': {'a': 100, 'b': 10, 'c': 10, 'd': 20},
   'nodes': {}}}}

-------